# Basic Prompt Structures Tutorial

## Overview

This tutorial focuses on two fundamental types of prompt structures:
1. Single-turn prompts
2. Multi-turn prompts (conversations)

We'll use OpenAI's GPT model and LangChain to demonstrate these concepts.

## Motivation

Understanding different prompt structures is crucial for effective communication with AI models. Single-turn prompts are useful for quick, straightforward queries, while multi-turn prompts enable more complex, context-aware interactions. Mastering these structures allows for more versatile and effective use of AI in various applications.

## Key Components

1. **Single-turn Prompts**: One-shot interactions with the language model.
2. **Multi-turn Prompts**: Series of interactions that maintain context.
3. **Prompt Templates**: Reusable structures for consistent prompting.
4. **Conversation Chains**: Maintaining context across multiple interactions.

## Method Details

We'll use a combination of OpenAI's API and LangChain library to demonstrate these prompt structures. The tutorial will include practical examples and comparisons of different prompt types.

## Setup

First, let's import the necessary libraries and set up our environment.

In [8]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

from dotenv import load_dotenv
load_dotenv()

#os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY') # OpenAI API key
# Initialize the language model
#llm = ChatOpenAI(model="gpt-4o-mini")

from select_llm import get_llm, get_supported_ollama_models
ollama_model = "llama3.2:1b"  # or None for OpenAI
# ollama_model = "tinyllama:latest"  # or None for OpenAI
#ollama_model = "deepseek-r1:7b"
llm = get_llm(ollama_model) #,force_port=12434)
if llm is None:
    models = get_supported_ollama_models()
    print("Available Ollama models:", models)

Using Ollama on Windows (port 11434).


## 1. Single-turn Prompts

Single-turn prompts are one-shot interactions with the language model. They consist of a single input (prompt) and generate a single output (response).

In [9]:
single_turn_prompt = "What are the three primary colors?"
print(llm.invoke(single_turn_prompt).content)

The three primary colors are:
1. Red
2. Blue
3. Yellow


Now, let's use a PromptTemplate to create a more structured single-turn prompt:

In [ ]:
structured_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Provide a brief explanation of {topic} and list its three main components."
)

chain = structured_prompt | llm
print(chain.invoke({"topic": "color theory"}).content)

## 2. Multi-turn Prompts (Conversations)

Multi-turn prompts involve a series of interactions with the language model, allowing for more complex and context-aware conversations.

In [ ]:
conversation = ConversationChain(
    llm=llm, 
    verbose=True,
    memory=ConversationBufferMemory()
)

print(conversation.predict(input="Hi, I'm learning about space. Can you tell me about planets?"))
print(conversation.predict(input="What's the largest planet in our solar system?"))
print(conversation.predict(input="How does its size compare to Earth?"))

In [ ]:
# Modern simple equivalent (no deprecated ConversationChain)
# This recreates the classic "Current conversation:\n... Human: ...\nAI:" prompt style
# (if conversiona prediction is selected and manually manages the running history.

red = "\033[91m"
blue = "\033[94m"
green = "\033[92m"
reset = "\033[0m"

SYSTEM_PREAMBLE = (
    "The following is a friendly conversation between a human and an AI. "
    "The AI is talkative and provides lots of specific details from its context. "
    "If the AI does not know the answer to a question, it truthfully says it does not know."
)

_history = ""

def conversation(user_prompt: str, predict_conversation: bool = False, verbose: bool = True) -> str:
    global _history

    formatted_prompt = (
        f"{SYSTEM_PREAMBLE}\n\n"
        f"Current conversation:\n"
        f"{_history}"
        f"Human: {user_prompt}\n"
        f"AI:"
    )

    print(f"{red}\n> Entering conversation...\n{reset}")
    if verbose:
        print(f"{red}Prompt after formatting:\n{reset}")
        print(f"{blue}{formatted_prompt}{reset}")
        print()
    else:
        print(f"{red}User Prompt:\n{reset}")
        print(f"{blue}{user_prompt}{reset}")
        print()

    # When predict_conversation=False, stop before generating any "Human:" line
    #response_text = llm.invoke(formatted_prompt, config={"stop": stop}).content
    #response_text = llm.invoke(formatted_prompt, stop=stop).content
    #response_text = llm.invoke(formatted_prompt, config={"configurable": {"stop": stop}}).content
    if predict_conversation:
        response_text = llm.invoke(formatted_prompt).content
    else:
        stop = ["\nHuman:"] if not predict_conversation else None
        response_text = llm.invoke(formatted_prompt, stop=stop).content


    _history += f"Human: {user_prompt}\nAI: {response_text}\n"

    print(f"{red}> Finished.\n{reset}")
    print(f"{red}Response:{reset}", f"{green}{response_text}{reset}")

    return response_text

predict_conversation = True
verbose = False
conversation("Hi, I'm learning about space. Can you tell me about planets?",predict_conversation=predict_conversation,verbose=verbose)
conversation("What's the largest planet in our solar system?",predict_conversation=predict_conversation,verbose=verbose)
conversation("How does its size compare to Earth?",predict_conversation=predict_conversation,verbose=verbose)


In [ ]:
# Newer method: RunnableWithMessageHistory (recommended instead of ConversationChain)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories.in_memory import ChatMessageHistory

# In-memory store for session histories (use a persistent store in production)
session_store = {}

def get_session_history(session_id: str):
    if session_id not in session_store:
        session_store[session_id] = ChatMessageHistory()
    return session_store[session_id]

# Same system message as the default ConversationChain
prompt = ChatPromptTemplate.from_messages([
    ("system", "The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

chain = prompt | llm
conversation_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
)

config = {"configurable": {"session_id": "space_conversation"}}
r1 = conversation_with_history.invoke({"question": "Hi, I'm learning about space. Can you tell me about planets?"}, config=config)
r2 = conversation_with_history.invoke({"question": "What's the largest planet in our solar system?"}, config=config)
r3 = conversation_with_history.invoke({"question": "How does its size compare to Earth?"}, config=config)

print(r1.content if hasattr(r1, "content") else r1)
print(r2.content if hasattr(r2, "content") else r2)
print(r3.content if hasattr(r3, "content") else r3)

Let's compare how single-turn and multi-turn prompts handle a series of related questions:

In [ ]:
# Single-turn prompts
prompts = [
    "What is the capital of France?",
    "What is its population?",
    "What is the city's most famous landmark?"
]

print("Single-turn responses:")
for prompt in prompts:
    print(f"Q: {prompt}")
    print(f"A: {llm.invoke(prompt).content}\n")

# Multi-turn prompts
print("Multi-turn responses:")
conversation = ConversationChain(llm=llm, memory=ConversationBufferMemory())
for prompt in prompts:
    print(f"Q: {prompt}")
    print(f"A: {conversation.predict(input=prompt)}\n")

## Conclusion

This tutorial has introduced you to the basics of single-turn and multi-turn prompt structures. We've seen how:

1. Single-turn prompts are useful for quick, isolated queries.
2. Multi-turn prompts maintain context across a conversation, allowing for more complex interactions.
3. PromptTemplates can be used to create structured, reusable prompts.
4. Conversation chains in LangChain help manage context in multi-turn interactions.

Understanding these different prompt structures allows you to choose the most appropriate approach for various tasks and create more effective interactions with AI language models.